In [6]:
NUMERIC = 1
ALPHA_NUMERIC = 2
BYTE = 4
KANJI = 8
MAX_BYTES = 14 #with version 1 and error correction level Medium
DATA_CODEWORD_LEN = 20
PAD_BYTES = [0XEC, 0X11]

In [7]:
def to_byte(value, bit_length):
    return format(value, f'0{bit_length}b')

In [ ]:
data_type = BYTE
char_count = 0

# string_input = input("Enter the data to be encoded(14 chars max)")
string_input = "HelloWorld"
char_count = len(string_input)
if char_count >14:
    raise RuntimeError("Invalid Input")

bit_stream = to_byte(data_type, 4)
bit_stream += to_byte(char_count, 8)

for x in string_input:
    bit_stream += to_byte(ord(x), 8)

#Adding terminators to signal end of data words, at max 4 bits
max_bits = MAX_BYTES*8          
rem_bits = min(4, max_bits - len(bit_stream))
bit_stream += '0'*rem_bits

while len(bit_stream) % 8 != 0: #Not needed in byte mode but may be needed in other modes to make the bit stream a multiple of 8
    bit_stream += '0'


data_codeword = [int(bit_stream[i:i+8], 2) for i in range(0, len(bit_stream), 8)] #Data word paired in groups of 8 and converted into integer

alternate = 1
while len(data_codeword) < DATA_CODEWORD_LEN: #Pad the message to the maximum limit
    data_codeword.append(PAD_BYTES[alternate%2])
    alternate += 1

print(data_codeword)

[64, 164, 134, 86, 198, 198, 245, 118, 247, 38, 198, 64, 17, 236, 17, 236, 17, 236, 17, 236]


In [9]:
a = "Hello World"
code_word = [ord(x) for x in a]
code_word.append(00)
print(code_word)
code_word.append(00)
print(len(code_word))

[72, 101, 108, 108, 111, 32, 87, 111, 114, 108, 100, 0]
13


In [ ]:
alpha = 0x02 #Alpha is an an element which can generate all elements of the field after being raiesd to some power
primitive = 0x11D #Primitive is irreductible polynomial that defines the Gallois Field.

log = [0]*256
antilog = [0]*512 #Since the operations are closed so there must be wraparound i.e. alpha^255 = alpha^0 = 1 so when there is antilog(log[a] + log[b]), antilog(log[a] + log[b] % 255) is not required since the values beyond 255 are kept the same as the initial values.

x = 1

for i in range(255):
    antilog[i] = x
    log[x] = i

    x = x<<1 #Left Shift by 1 i.e. multiply by 2 i.e. alpha, microprocessor nostalgia
    
    if x & 0x100: #If there is 9th bit then it is beyond GF(256)
        x = x^primitive #EXOR is same as addition/substraction, later in this case, in GF(2)

for i in range(255, 512):
    antilog[i] = antilog[i-255]

In [ ]:
def gf_addition(a, b):
    return a^b

def gf_multiplication(a, b):
    if a==0 or b==0:
        return 0
    return antilog[log[a] + log[b]]

def gf_division(a, b):
    if a==0:
        return 0
    if b==0:
        raise ZeroDivisionError()
    return antilog[log[a] - log[b]]

def poly_mul(a, b):
    result = [0]*(len(a)+len(b)-1)
    for i in range(len(a)):
        for j in range(len(b)):
            result[i+j] ^= gf_multiplication(a[i], b[j])
    return result

def generate_generator(num_ec):
    g = [1]
    for i in range(num_ec):
        g = poly_mul(g, [1, antilog[i]])  # (x-a^i) is written as [1, antilog[i] i.e. a^i] and +/- are same in GF(256)
    return g

In [ ]:
def poly_div(dividend, divisor): #We perform polynomial long division after which the remainder will be the error correction codeword
    divi = list(dividend)
    for i in range(len(dividend) - len(divisor)+1):
        coeff = divi[i]
        if coeff!=0:
            for j in range(len(divisor)):
                divi[i+j] ^= gf_multiplication(divisor[j], coeff) 
                '''This is in place division where the dividend is modified in each operation 
                    divisor*highest_coeff_of_dividend is substracted from dividend taking the first terms equal to the number of terms of divisor'''
    remainder = divi[-(len(dividend)-1):]  #The last remaining terms with degree, at most, one less than divisor
    return remainder